In [1]:
import pandas as pd
import numpy as np
from scipy.stats import truncnorm
import re
import random
import geopandas as gpd
from numba import jit, njit
from shapely import MultiPoint, Point
from shapely.ops import nearest_points

In [2]:
dwellings_file = './Output/dd_2006_snapped.csv'
# jobs_file = './jj_2006_2138_snapped.csv'
persons_file = './Output/TESTPERSONS.csv'

In [9]:
job_type_indexes = ["Agri","Mnft","Util","Cons","Retl","Trns","Finc","Rlst","Admn","Serv"]

In [32]:
def convert_job_type_to_int(job):
    jobType = job[3]
    if jobType in job_type_indexes:
        job[3] = job_type_indexes.index(jobType)
    else:
        job[3] = -1
    return job

In [31]:
dwellings = pd.read_csv(dwellings_file)
d_cols = dwellings.columns
d = lambda x: d_cols.get_loc(x)
dwellings:np.ndarray = dwellings.values

persons = pd.read_csv(persons_file)
# nats = persons['nationality']
p_cols = persons.columns
p = lambda x: p_cols.get_loc(x)
persons:np.ndarray = persons[p_cols].values
persons = persons.astype(np.float32)

households_df = pd.DataFrame(columns=['id', 'dwelling', 'hhSize', 'autos'], dtype=int)
h_cols = households_df.columns
h = lambda x: h_cols.get_loc(x)


jobs = pd.read_csv('./Output/jj_2006_2138_snapped.csv')
j_cols = jobs.columns
jobs = jobs.values


In [33]:
np.apply_along_axis(convert_job_type_to_int, 1, jobs)

array([[0, 24660061, -1, ..., 193044.04741146124, 32400, 28800],
       [1, 24660061, -1, ..., 193044.04741146124, 32400, 28800],
       [2, 24660061, -1, ..., 193044.04741146124, 32400, 28800],
       ...,
       [1699442, 24590189, -1, ..., 210073.2706329787, 32400, 28800],
       [1699443, 24590189, -1, ..., 210084.9938897257, 32400, 28800],
       [1699444, 24590189, -1, ..., 210073.2706329787, 32400, 28800]],
      dtype=object)

In [4]:
@jit(nopython=True)
def findPartner(n, person):
    for i in range(n, persons.shape[0]):
        if persons[i][4]==1 and persons[i][1]==-1 and \
                                        abs(person[2] - persons[i][2]) < 10:
            # print(person[0], 'married to ', persons[i][0])
            return i # retur person index
    return None

In [5]:
@jit(nopython=True)
def findRoommate(n, person):
    for i in range(n, persons.shape[0]):
        if persons[i][4]==0 and persons[i][1]==-1 and \
                                        abs(person[2] - persons[i][2]) < 10:
            return i # return person index
    return None

In [6]:
@jit(nopython=True)
def firstLonelyChild(n):
    for i in range(n, persons.shape[0]):
        if persons[i][1]==-1 and persons[i][4]==2:
            lastIndex = i
            return int(persons[i][0])

In [7]:
@jit(nopython=True)
def nextUnhousedPerson(n):
    for i in range(n, persons.shape[0]):
        if persons[i][1]==-1 and persons[i][4] != 2:
            return int(persons[i][0])

In [35]:
@jit(nopython=True)
def getNextJob(n):
    for i in range(n, jobs.shape[0]):
        if jobs[i][2] == -1:
            return jobs[i][0]

<h1>GIVE PERSONS JOBS</h1>

In [ ]:
lastJob = 0
def assign_persons_jobs(person):
    if persons[]
    person[7] = getNextJob(lastJob)

In [ ]:
np.apply_along_axis(assign_persons_jobs, 1, persons)

HOUSEHOLD SYNTHESIS

for each dwelling
- create a household
  - has either a single person and 0-1 kids
  - OR married couple with 0-2 kids
- 

In [8]:
households = np.empty([dwellings.shape[0],4])

In [9]:
householdNumber = 0
personNum = 0
lastChildIndex = 0
err = 1 # exhausted dwellings, extra people
for i, dwelling in enumerate(dwellings): # for each dwelling
    hhId = householdNumber
    if persons[personNum][1]==-1 and persons[personNum][4] != 2: # if current person is unhoused and not a child
        person = persons[personNum] # random.randint(0, len(persons)-1)
    else:
        next = nextUnhousedPerson(personNum) # skip to the index of the next unhoused person to save time
        if next == None:
            err=2 # exhausted people, extra dwellings
            break
        person = persons[next]
        personNum = int(person[0])
        
    persons[personNum][p('hhId')] = hhId
    hhSize = 1
    numKids=0
    if person[p('relationship')]==1: # married
        partner = findPartner(int(personNum), person)
        if partner == None: # can't find partner for person
            persons[personNum][p('relationship')]=0 # set person to single
            numKids = random.randint(0,1)
            hhSize += numKids
        else:  
            persons[int(partner)][1] = hhId
            numKids = random.randint(0,3)
            hhSize += numKids+1

        for j in range(numKids):
            child_ind = firstLonelyChild(lastChildIndex)
            if child_ind != None:
                persons[child_ind][1] = hhId
                lastChildIndex = child_ind
        

    elif person[p('relationship')]==0: # single
        numKids=random.randint(0,1)
        numRoommates = random.randint(0,2)
        if numKids>0:
            child_ind = firstLonelyChild(lastChildIndex)
            if child_ind != None:
                persons[child_ind][1] = hhId
                lastChildIndex = child_ind
        
        if numRoommates>0:
            for rm in range(numRoommates):
                roommate_ind = findRoommate(personNum+1, person)
                if roommate_ind != None:
                    persons[roommate_ind][1] = hhId
        hhSize += numKids+numRoommates

    # households = np.r_[households, [[hhId, dwelling[d('id')], hhSize, random.randint(0,hhSize-numKids)]]]
    households[householdNumber][0] = hhId
    households[householdNumber][1] = dwelling[d('id')]
    households[householdNumber][2] = hhSize
    households[householdNumber][3] = random.randint(0,hhSize-numKids)
    householdNumber+=1
    personNum += 1
    dwellings[i][d('hhId')] = hhId

    if i%10000==0:
        print(i)
    # if i%150000==0 and i!=0:
    #     num=int(i/150000)
    #     print('writing to file:', num)
    #     pd.DataFrame(data=households, columns=h_cols).astype(int).to_csv('./Output/hh_2006_'+str(num)+'.csv', index=False)
    #     households = np.empty([0,4])

if err==1:
    print("Exhausted Dwellings, extra people with no dwelling")
    print('At person', personNum)
elif err==2:
    print("All poeple housed, extra dwellings available")


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
All poeple housed, extra d

In [18]:
households = households[:householdNumber]

In [10]:
pd.DataFrame(data=households, columns=h_cols).astype(int).to_csv('./Output/hh_2006.csv', index=False)

In [27]:
pd.DataFrame(data=dwellings, columns=d_cols).to_csv('./Output/dd_2006_snapped_linked.csv', index=False)

GENERATE NATIONALITIES

In [24]:
tmp = """Canadian 1,670,655
French 870,245
Italian	279,800
Irish 239,460
English 138,320
Haitian 132,255
Scottish 124,130
Chinese	108,775
First Nations 101,915
Québécois 92,115
German 86,025
Algerian 84,585
Moroccan 77,450
Spanish	68,600
Greek 66,395
Lebanese 68,765
Polish 64,895
Portuguese 56,405
Russian	49,275
East Indian	48,485
Romanian 47,980
Vietnamese 38,660
Filipino 35,685
Ukrainian 35,050
Belgian	31,840"""

countries = tmp.replace('\t', ' ').replace(',', '').split('\n')
nationalities = {}
total_nationalities_pop = 0
for i in countries:
    ind = re.search('[0-9]', i).span()[0]
    nationalities[i[:ind-1]] = 0.0
    total_nationalities_pop += int(i[ind:])

for i in countries:
    ind = re.search('[0-9]', i).span()[0]
    nationalities[i[:ind-1]] = int(i[ind:])/total_nationalities_pop

print(nationalities)
nationalities_list = random.choices(list(nationalities.keys()), weights=list(nationalities.values()),k=persons.shape[0])

{'Canadian': 0.3617886574998944, 'French': 0.18845588720950504, 'Italian': 0.06059208296654334, 'Irish': 0.05185625513641339, 'English': 0.029953884617341938, 'Haitian': 0.028640478673124335, 'Scottish': 0.026880969473327465, 'Chinese': 0.023555767779434425, 'First Nations': 0.022070200627359773, 'Québécois': 0.0199479618386817, 'German': 0.018629142020003184, 'Algerian': 0.018317302851054568, 'Moroccan': 0.01677218307990987, 'Spanish': 0.014855671520746508, 'Greek': 0.014378167793293942, 'Lebanese': 0.014891403092188537, 'Polish': 0.014053335325639136, 'Portuguese': 0.012214783558712927, 'Russian': 0.010670746562460412, 'East Indian': 0.010499668129495546, 'Romanian': 0.010390307865385095, 'Vietnamese': 0.00837201546635656, 'Filipino': 0.00772776440550786, 'Ukrainian': 0.007590251994200658, 'Belgian': 0.006895110513419371}


In [26]:
outputPersons = pd.DataFrame(data=persons, columns=p_cols)
outputPersons['nationality'] = nationalities_list
outputPersons.to_csv('./Output/pp_2006_linked.csv', index=False)